In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from df_plot import plot_df 
from info_value_to_net import info_value_to_net
from get_info_values import mutual_info_measures, cond_kl_divergence
from save_info_values import save_info_values

from preprocessing import preprocessing

np.seterr(divide='ignore', invalid = 'ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

### Designing a national strategy

#### Option 1: Perform FIT to the n patients with higher utility measured by the model, being n the number of people that the current screening strategy covers (that is, the number of people above the age of 54)

In [29]:
from preprocessing import preprocessing

flag = True

if not flag:
    df_test = pd.read_csv("private/df_2016.csv")
    df_test = preprocessing(df_test)

    df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

    df_utilities_2016 = pd.read_csv("outputs/utilities_2016.csv")

else:
    model_type = "linear"

    net = pysmile.Network()
    net.read_file(f"decision_models/DM_screening_rel_pcmi_{model_type}.xdsl")

    df_test = pd.read_csv("private/df_2016.csv")
    df_test = preprocessing(df_test)

    df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

    # Just keep variables that influence the decision
    # df_test.drop(columns = ["Hyperchol_", "Hypertension", "Diabetes", "SES", "Anxiety", "Depression"], inplace = True)

    y = np.array(list(df_test["CRC"]*1))

    vars1 = np.array(["No scr", "gFOBT", "FIT", "Blood_test", "sDNA", "CTC", "CC", "Colonoscopy"])

    df_utilities_2016 = pd.DataFrame([], columns = vars1)

    for i in range(df_test.shape[0]):
        sample = df_test.iloc[i].drop(labels = ["CRC"])
        sample_dict = sample.to_dict() 

        net.clear_all_evidence()

        for key, value in sample.items():
            if type(value) == np.bool_:
                net.set_evidence(key, bool(value))
            else:
                net.set_evidence(key, value)

        net.update_beliefs()

        arr = np.array(net.get_node_value("U"))

        U = np.concatenate((arr[::2], [arr[1]]), axis = 0)

        row = pd.DataFrame(U.reshape(1,8), columns=vars1)
        df_utilities_2016 = pd.concat([df_utilities_2016, row], ignore_index = True)

    df_utilities_2016 = pd.concat( [df_utilities_2016, df_test["CRC"] ] , axis = 1)
    df_utilities_2016.to_csv("outputs/utilities_2016_fulldataset.csv", index = False)


C:\Users\danie\AppData\Local\Temp\ipykernel_25940\2674396849.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_utilities_2016 = pd.concat([df_utilities_2016, row], ignore_index = True)


In [30]:
df_test

,Age,Sex,BMI,Alcohol,Smoking,PA,Depression,Anxiety,Diabetes,Hypertension,Hyperchol_,SD,SES,CRC
0,age_3_young_adult,M,bmi_3_overweight,low,sm_1_not_smoker,PA_2,False,False,False,False,False,SD_2_normal,ses_1,False
1,age_4_adult,M,bmi_3_overweight,low,sm_2_smoker,PA_1,False,False,False,False,False,SD_2_normal,ses_1,False
2,age_5_old_adult,M,bmi_4_obese,low,sm_3_ex_smoker,PA_2,False,False,False,True,False,SD_2_normal,ses_1,False
3,age_4_adult,M,bmi_2_normal,low,sm_1_not_smoker,PA_2,False,False,False,False,False,SD_2_normal,ses_1,False
4,age_3_young_adult,M,bmi_3_overweight,low,sm_1_not_smoker,PA_1,False,False,False,False,False,SD_2_normal,ses_1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339702,age_4_adult,M,bmi_3_overweight,low,sm_1_not_smoker,PA_1,False,False,False,False,True,SD_2_normal,ses_1,False
339703,age_4_adult,M,bmi_2_normal,low,sm_1_not_smoker,PA_2,False,False,False,False,False,SD_2_normal,ses_1,False
339704,age_4_adult,M,bmi_2_normal,high,sm_1_not_smoker,PA_2,False,False,False,False,False,SD_2_normal,ses_1,False
339705,age_2_young,M,bmi_3_overweight,low,sm_1_not_smoker,PA_2,False,False,False,False,False,SD_2_normal,ses_1,False


In [31]:
df_selected_current_strategy = df_test[df_test["Age"] == "age_5_old_adult"]
tot_num_patients = df_selected_current_strategy.shape[0]   
print("------------------------")
print("Number of selected patients by current strategy is: ", tot_num_patients)
print("Number of patients with CRC by current strategy is: ", df_selected_current_strategy["CRC"].sum())
print("Percentage of total patients with CRC detected:",  (df_selected_current_strategy["CRC"].sum() / df_test["CRC"].sum()).round(4) * 100, "%")

print("------------------------")
print("Selecting the same number of patients for the new strategy, that is, ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients].shape[0])
print("Number of patentes with CRC by new strategy is: ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum())
print("Percentage of total patients with CRC detected:", (df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() / df_test["CRC"].sum()).round(4) * 100, "%")

print("------------------------")
print("This is an improvement of ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() - df_selected_current_strategy["CRC"].sum(), "patients")
print("A proportional improvement of ", ((df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() - df_selected_current_strategy["CRC"].sum()) / df_selected_current_strategy["CRC"].sum()).round(4) * 100, "%")

------------------------
Number of selected patients by current strategy is:  49074
Number of patients with CRC by current strategy is:  107
Percentage of total patients with CRC detected: 49.08 %
------------------------
Selecting the same number of patients for the new strategy, that is,  49074
Number of patentes with CRC by new strategy is:  109
Percentage of total patients with CRC detected: 50.0 %
------------------------
This is an improvement of  2 patients
A proportional improvement of  1.87 %


**Comment:** Using the full dataset with information on the medical conditions, we improve in detecting CRC as now our target population has 2 more actual CRC positive patients. This is a small improvement (around 2%) but it might mean 2 lives saved. On the contrary, using the selected variables for the decision node (that is, the ones that are simpler and supposedly do not require a GP check) does not change the number of CRC detected patients.

-> **Questions:**
- Does the target population even change at all for the second case?
- What is the effect of adding each of the medical conditions separately?

#### Option 2: Apply new national strategy with N FITs and M colonoscopies available.

In [4]:
N_FIT = 2000
M_COL = 600

In [4]:
import numpy as np
import pandas as pd

def simulate_test_results(sensitivity_scr, specificity_scr,
                           sensitivity_col, specificity_col, y_crc):
    """
    Simulate test results based on sensitivity, specificity, and actual number of patients
    with and without the disease.
    
    Parameters:
    - sensitivity (float): Sensitivity of the test (true positive rate)
    - specificity (float): Specificity of the test (true negative rate)
    - num_with_disease (int): Number of patients who have the disease
    - num_without_disease (int): Number of patients who do not have the disease
    
    Returns:
    - pandas DataFrame: A DataFrame with the simulated test results, true conditions, and test outcomes.
    """
    

    num_with_disease = y_crc.sum()
    num_without_disease = len(y_crc) - num_with_disease

    # Step 1: Create a list of patients with and without the disease
    
    # Step 2: Simulate test results
    scr_results = []
    col_results = []
    
    for y in y_crc:
        if y == 1:
            # Patient has the disease, test is positive with probability = sensitivity
            scr_result = np.random.choice([1, 0], p=[sensitivity_scr, 1 - sensitivity_scr])
        else:
            # Patient does not have the disease, test is negative with probability = specificity
            scr_result = np.random.choice([0, 1], p=[specificity_scr, 1 - specificity_scr])

        scr_results.append(scr_result)
    
    # Step 3: Create a DataFrame to store the results
    df_scr = pd.DataFrame({
        'Condition': y_crc,    # True condition of the patient
        'TestResult': scr_results  # Simulated test result
    })


    # For FIT positives, perform colonoscopy:
    FIT_positives = df_scr[df_scr["TestResult"] == 1]

    conditions = FIT_positives["Condition"].to_list()

    col_results = []

    for condition in conditions:
        if condition == 1:
            # Patient has the disease, test is positive with probability = sensitivity
            col_result = np.random.choice([1, 0], p=[sensitivity_col, 1 - sensitivity_col])
        else:
            # Patient does not have the disease, test is negative with probability = specificity
            col_result = np.random.choice([0, 1], p=[specificity_col, 1 - specificity_col])

        col_results.append(col_result)

    # Step 5: Create a DataFrame to store the results
    df_col = pd.DataFrame({
        'Condition': conditions,    # True condition of the patient
        'TestResult': col_results  # Simulated test result
    })

    
    
    return df_scr, df_col

In [5]:
def output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = False):

    # Add columns to indicate true positives, false positives, etc.
    df_scr['TruePositive'] = (df_scr['Condition'] == 1) & (df_scr['TestResult'] == 1)
    df_scr['FalsePositive'] = (df_scr['Condition'] == 0) & (df_scr['TestResult'] == 1)
    df_scr['TrueNegative'] = (df_scr['Condition'] == 0) & (df_scr['TestResult'] == 0)
    df_scr['FalseNegative'] = (df_scr['Condition'] == 1) & (df_scr['TestResult'] == 0)
    
    # Step 4: Calculate confusion matrix components
    TP_scr = df_scr['TruePositive'].sum()
    FP_scr = df_scr['FalsePositive'].sum()
    TN_scr = df_scr['TrueNegative'].sum()
    FN_scr = df_scr['FalseNegative'].sum()
    
    # Create confusion matrix
    confusion_matrix_scr = pd.DataFrame({
        'Predicted Negative': [TN_scr, FN_scr],
        'Predicted Positive': [FP_scr, TP_scr]
    }, index=['Actual Negative', 'Actual Positive'])


    FIT_positives = df_scr[df_scr["TestResult"] == 1]
    patient_data = df_scr["Condition"]

    if verbose:
        print("Number of patients considered: ", patient_data.shape[0])
        print(f"Cost of screening: {cost_scr*(patient_data.shape[0])} €")
        print("Number of FIT positives: ", FIT_positives.shape[0])
        print("Number of colonoscopies to be done: ", FIT_positives.shape[0])
        print(f"Cost of colonoscopy program: {cost_col*FIT_positives.shape[0]} €")

    
    # Add columns to indicate true positives, false positives, etc.
    df_col['TruePositive'] = (df_col['Condition'] == 1) & (df_col['TestResult'] == 1)
    df_col['FalsePositive'] = (df_col['Condition'] == 0) & (df_col['TestResult'] == 1)
    df_col['TrueNegative'] = (df_col['Condition'] == 0) & (df_col['TestResult'] == 0)
    df_col['FalseNegative'] = (df_col['Condition'] == 1) & (df_col['TestResult'] == 0)

    # Step 6: Calculate confusion matrix components
    TP_col = df_col['TruePositive'].sum()
    FP_col = df_col['FalsePositive'].sum()
    TN_col = df_col['TrueNegative'].sum()
    FN_col = df_col['FalseNegative'].sum()

    # Create confusion matrix
    confusion_matrix_col = pd.DataFrame({
        'Predicted Negative': [TN_col, FN_col],
        'Predicted Positive': [FP_col, TP_col]
    }, index=['Actual Negative', 'Actual Positive'])

    total_cost = cost_scr*df_scr["Condition"].shape[0] + cost_col*FIT_positives.shape[0]

    if verbose:
        print("Number of CRC true positive cases detected by colonoscopy: ", TP_scr)
        print("Number of false positives by colonoscopy: ", FP_scr)
        print(f"Total cost of screening and colonoscopy: {total_cost} €")
        print("Proportion of total CRC cases in the whole population detected by the method: ", TP_scr / df_test["CRC"].sum())
        print("Proportion of cases in the high-risk target population detected by the method: ", TP_scr / y.sum())

    combined_confusion_matrix = pd.DataFrame({
        'Predicted Negative': [TN_scr + TN_col, FN_scr + FN_col],
        'Predicted Positive': [FP_col, TP_col]
    }, index=['Actual Negative', 'Actual Positive'])

    # Calculate sensitivity and specificity using the combined confusion matrix
    sensitivity = TP_col / (TP_col + FN_col + FN_scr)
    specificity = (TN_scr + TN_col) / (TN_scr +TN_col + FP_col)
    PPV = TP_col / (TP_col + FP_col)
    NPV = (TN_scr + TN_col) / (TN_scr + TN_col + FN_scr + FN_col)

    metrics = {
        "sensitivity": sensitivity,
        "specificity": specificity,
        "PPV": PPV,
        "NPV": NPV
    }
    
    return confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics
    

**Old screening strategy**

In [6]:
df_test = pd.read_csv("private/df_2016.csv")
df_test = preprocessing(df_test)

df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

In [7]:
sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

# Select old adults as high-risk target population
y = np.array(list(df_test[df_test["Age"] == "age_5_old_adult"]["CRC"]*1))

df_scr, df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y)

In [8]:
confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = True)

Number of patients considered:  49074
Cost of screening: 703721.16 €
Number of FIT positives:  1734
Number of colonoscopies to be done:  1734
Cost of colonoscopy program: 1734000 €
Number of CRC true positive cases detected by colonoscopy:  75
Number of false positives by colonoscopy:  1659
Total cost of screening and colonoscopy: 2437721.16 €
Proportion of total CRC cases in the whole population detected by the method:  0.3440366972477064
Proportion of cases in the high-risk target population detected by the method:  0.7009345794392523


In [9]:
confusion_matrix_scr

,Predicted Negative,Predicted Positive
Actual Negative,47308,1659
Actual Positive,32,75


In [10]:
confusion_matrix_col

,Predicted Negative,Predicted Positive
Actual Negative,1646,13
Actual Positive,1,74


In [11]:
combined_confusion_matrix

,Predicted Negative,Predicted Positive
Actual Negative,48954,13
Actual Positive,33,74


In [12]:
metrics

{'sensitivity': 0.6915887850467289,
 'specificity': 0.9997345150815855,
 'PPV': 0.8505747126436781,
 'NPV': 0.9993263518892768}

**New screening strategy**

Say we have 10,000 FITs and 2,000 colonoscopies. We would take the first 2,000 patients with highest utility based on our model and perform FIT on them. If positive, then proceed with a colonoscopy.

In [13]:
df_utilities_2016 = pd.read_csv("utilities_2016.csv")
df_utilities_2016_sorted = df_utilities_2016.sort_values(by = "FIT", ascending = False)

In [14]:
sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

# Select as high-risk target population the top 20000 patients according to utility in our model
y = np.array(list(df_utilities_2016_sorted["CRC"]*1))[:49074]

df_scr,  df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y)

In [15]:
confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = True)

Number of patients considered:  49074
Cost of screening: 703721.16 €
Number of FIT positives:  1667
Number of colonoscopies to be done:  1667
Cost of colonoscopy program: 1667000 €
Number of CRC true positive cases detected by colonoscopy:  83
Number of false positives by colonoscopy:  1584
Total cost of screening and colonoscopy: 2370721.16 €
Proportion of total CRC cases in the whole population detected by the method:  0.38073394495412843
Proportion of cases in the high-risk target population detected by the method:  0.7614678899082569


In [16]:
confusion_matrix_scr

,Predicted Negative,Predicted Positive
Actual Negative,47381,1584
Actual Positive,26,83


In [17]:
confusion_matrix_col

,Predicted Negative,Predicted Positive
Actual Negative,1573,11
Actual Positive,3,80


In [18]:
combined_confusion_matrix

,Predicted Negative,Predicted Positive
Actual Negative,48954,11
Actual Positive,29,80


In [19]:
metrics

{'sensitivity': 0.7339449541284404,
 'specificity': 0.9997753497396099,
 'PPV': 0.8791208791208791,
 'NPV': 0.999407957862932}

### Simulation

In [21]:
n_samples = 200

sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

total_cost_old_strat = 0
total_cost_new_strat = 0  

metrics_tot_old = np.array([0.0,0.0,0.0,0.0])
metrics_tot_new = np.array([0.0,0.0,0.0,0.0])

# Old strategy
# Select old adults as high-risk target population
y_old = np.array(list(df_test[df_test["Age"] == "age_5_old_adult"]["CRC"]*1))
patients_selected = y_old.shape[0]

# New strategy
# Select as high-risk target population the top --- patients according to utility in our model
y_new = np.array(list(df_utilities_2016_sorted["CRC"]*1))[:patients_selected]

for i in range(n_samples):

    # -----
    df_scr,  df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y_old)
    confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = False)

    if i == 0:
        metrics_tot_old = np.array([*metrics.values()])
    else:
        metrics_tot_old = np.concatenate((metrics_tot_old, np.array([*metrics.values()])), axis = 0)

    total_cost_old = cost_scr*df_scr.shape[0] + cost_col*df_col.shape[0]
    total_cost_old_strat += total_cost_old


    # -----
    df_scr,  df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y_new)
    confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = False)

    if i == 0:
        metrics_tot_new = np.array([*metrics.values()])
    else:
        metrics_tot_new = np.concatenate((metrics_tot_new, np.array([*metrics.values()])), axis = 0)

    total_cost_new = total_cost = cost_scr*df_scr.shape[0] + cost_col*df_col.shape[0]
    total_cost_new_strat += total_cost_new

    if (i+1) % 20 == 0:
        print(f"Sample {i+1} done")


metrics_tot_old = metrics_tot_old.reshape(n_samples, 4)
metrics_tot_new = metrics_tot_new.reshape(n_samples, 4)

print("Average metrics for old strategy: ", metrics_tot_old.mean(axis=0))
print("Std for estimates: " , metrics_tot_old.std(axis=0))
print("Average cost of old strategy: ", total_cost_old_strat / n_samples)

print("Average metrics for new strategy: ", metrics_tot_new.mean(axis=0), "(+/-" , metrics_tot_new.std(axis=0), ")")
print("Std for estimates: " , metrics_tot_new.std(axis=0))
print("Average cost of new strategy: ", total_cost_new_strat / n_samples)

Sample 20 done
Sample 40 done
Sample 60 done
Sample 80 done
Sample 100 done
Sample 120 done
Sample 140 done
Sample 160 done
Sample 180 done
Sample 200 done
Average metrics for old strategy:  [0.71070093 0.99965834 0.82071877 0.99936803]
Std for estimates:  [4.55582075e-02 8.34731953e-05 3.77652862e-02 9.94610607e-05]
Average cost of old strategy:  2447711.16000001
Average metrics for new strategy:  [0.70779817 0.99965782 0.82264662 0.99934975] (+/- [4.23983260e-02 8.01576064e-05 3.50670101e-02 9.42864195e-05] )
Std for estimates:  [4.23983260e-02 8.01576064e-05 3.50670101e-02 9.42864195e-05]
Average cost of new strategy:  2448911.16000001


In [30]:
metrics_tot_old = np.concatenate((metrics_tot_old, np.array([*metrics.values()])), axis = 0)
metrics_tot_old.reshape(7,4)

array([[357.39252336, 499.82835379, 410.30531622, 499.68847327],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427],
       [  0.65137615,   0.99965281,   0.80681818,   0.99922427]])

In [34]:
metrics_tot_old.reshape(7,4).mean(axis = 0)

array([51.61439718, 72.26089581, 59.30660362, 72.24054555])

In [123]:
n_samples = 100

sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

total_cost_old_strat = 0
total_cost_new_strat = 0  

metrics_tot_old = np.array([0.0,0.0,0.0,0.0])
metrics_tot_new = np.array([0.0,0.0,0.0,0.0])


def run_iteration(i, df_test, df_utilities_2016_sorted, sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, cost_scr, cost_col):

    # Old strategy
    # Select old adults as high-risk target population
    y = np.array(list(df_test[df_test["Age"] == "age_5_old_adult"]["CRC"]*1))
    patients_selected = y.shape[0]

    df_scr,  df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y)
    confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics_old = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = False)

    total_cost_old_value = cost_scr*df_scr.shape[0] + cost_col*df_col.shape[0]


    # New strategy
    # Select as high-risk target population the top --- patients according to utility in our model
    y = np.array(list(df_utilities_2016_sorted["CRC"]*1))[:patients_selected]

    df_scr,  df_col = simulate_test_results(sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, y)
    confusion_matrix_scr, confusion_matrix_col, combined_confusion_matrix, total_cost, metrics_new = output_test_results(df_scr, cost_scr, df_col, cost_col, verbose = False)

    metrics_tot_new += np.array([*metrics.values()])

    total_cost_new_value = total_cost = cost_scr*df_scr.shape[0] + cost_col*df_col.shape[0]


    return {
        "metrics_old": metrics_old,
        "total_cost_old": total_cost_old_value,
        "metrics_new": metrics_new,
        "total_cost_new": total_cost_new_value
    }



In [124]:
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed

# Parameters
n_samples = 100  # Define your number of samples here
metrics_tot_old = np.zeros(4)  # Adjust based on your metrics' size
metrics_tot_new = np.zeros(4)  # Adjust based on your metrics' size
total_cost_old_strat = 0
total_cost_new_strat = 0

# Create a process pool
with ProcessPoolExecutor() as executor:
    # Submit all iterations
    futures = {executor.submit(run_iteration, i, df_test, df_utilities_2016_sorted, sensitivity_scr, specificity_scr, sensitivity_col, specificity_col, cost_scr, cost_col): i for i in range(n_samples)}

    for future in as_completed(futures):
        result = future.result()
        metrics_tot_old += np.array([*result["metrics_old"].values()])
        total_cost_old_strat += result["total_cost_old"]
        
        metrics_tot_new += np.array([*result["metrics_new"].values()])
        total_cost_new_strat += result["total_cost_new"]
        
        sample_index = futures[future]
        if (sample_index + 1) % 20 == 0:
            print(f"Sample {sample_index} done")

# Final results
print("Average metrics for old strategy: ", metrics_tot_old / n_samples)
print("Average cost of old strategy: ", total_cost_old_strat / n_samples)

print("Average metrics for new strategy: ", metrics_tot_new / n_samples)
print("Average cost of new strategy: ", total_cost_new_strat / n_samples)